In [62]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Input, Dense,SpatialDropout1D
from keras.layers import LSTM, Embedding
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from nltk.corpus import stopwords

In [5]:
#df=pd.read_csv('https://raw.githubusercontent.com/kushalchauhan98/bcn-cnn-text-classification/master/hm_train.csv')
#df.to_csv('hm_train.csv')
df=pd.read_csv('hm_train.csv')
#https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17

In [80]:
filter=df['cleaned_hm'].str.len()<=100
df=df[filter]
#rint(df)
#y=df['predicted_category']

y = pd.get_dummies(df['predicted_category']).values
print('Shape of label tensor:', y.shape)

LABELS=sorted(df['predicted_category'].unique())
#LABELS=['achievement','affection','bonding','enjoy_the_moment','exercise','leisure','nature']

print(pd.get_dummies(df['predicted_category']))
print(LABELS)

Shape of label tensor: (43354, 7)
       achievement  affection  bonding  enjoy_the_moment  exercise  leisure  \
0                0          1        0                 0         0        0   
1                0          1        0                 0         0        0   
2                0          0        0                 0         1        0   
4                0          1        0                 0         0        0   
5                0          0        0                 0         0        1   
...            ...        ...      ...               ...       ...      ...   
60315            0          0        0                 0         0        1   
60316            0          0        1                 0         0        0   
60317            0          0        1                 0         0        0   
60318            1          0        0                 0         0        0   
60319            1          0        0                 0         0        0   

       nature  
0

In [81]:


# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

stop_words = set(stopwords.words('english'))

def filter_stop_words(train_sentences, stop_words):
    for i, sentence in enumerate(train_sentences):
        new_sent = [word for word in sentence.split() if word not in stop_words]
        train_sentences[i] = ' '.join(new_sent)
    return train_sentences

stop_words = set(stopwords.words("english"))
sentences = filter_stop_words(df['cleaned_hm'].values, stop_words)

#word_tokens = word_tokenize(example_sent)
#filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
#filtered_sentence = []
#for w in word_tokens:
#    if w not in stop_words:
#        filtered_sentence.append(w)
        
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

X = tokenizer.texts_to_sequences(sentences)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.10, random_state = 42)



Shape of data tensor: (43354, 250)


In [82]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(len(LABELS), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Train on 35116 samples, validate on 3902 samples
Epoch 1/5
   64/35116 [..............................] - ETA: 8:58 - loss: 1.9389 - accuracy: 0.2500

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
#def GetCategory(pred,LABELS):
#    returnVal=[]
#    curr_index=0
#    curr_val=0
#    for item in pred:
#        val=max(item)
#        index=list(item).index(val)
#        #print(val,LABELS[index])
#        if val>curr_val:
#            curr_index=index
#            curr_val=val
            
       
for key,item in df.iterrows():
    data=[]
    data.append(item['cleaned_hm'])
    sentences = filter_stop_words(data, stop_words)
    #seq = tokenizer.texts_to_sequences(data)
    seq = tokenizer.texts_to_sequences(sentences)
    padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
    pred = model.predict(padded)
    print(pred)
    category=LABELS[np.argmax(pred)]
    #category=GetCategory(pred,LABELS)
    print(data, "[predicted category]", category, "[actual category]",item['predicted_category'])
    #break
    #print(pred, LABELS[np.argmax(pred)])